In [ ]:
# Import required libraries
import requests
import datetime
import ipywidgets as widgets
import plotly.graph_objects as go
from IPython.display import display, clear_output
import pandas as pd
from datetime import datetime, timedelta

# Initialize global variables
token = None
results = {}
samples = []
depositions = []
jvs = []

# Create widgets first
username = widgets.Text(description='Username:', placeholder='Enter your email')
password = widgets.Password(description='Password:', placeholder='Enter password')
login_button = widgets.Button(description='Login')
login_output = widgets.Output()

start_date = widgets.DatePicker(
    description='Start Date:',
    value=datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) - timedelta(days=70)
)
end_date = widgets.DatePicker(
    description='End Date:',
    value=datetime.now().replace(hour=23, minute=59, second=59, microsecond=999999)
)
date_container = widgets.HBox([start_date, end_date])
device_toggles = widgets.VBox([])
plot_output = widgets.Output()

# Set up base URL and sample entry type
url_base = "http://elnserver.lti.kit.edu"
url = f"{url_base}/nomad-oasis/api/v1"
sample_entry = "peroTF_Sample"

def get_token_interactive(url, username, password):
    """Get authentication token using provided credentials"""
    try:
        response = requests.get(f'{url}/auth/token', 
                              params=dict(username=username, password=password))
        if response.status_code == 401:
            raise Exception(response.json()["detail"])
        return response.json()['access_token']
    except Exception as e:
        raise Exception(f"Authentication failed: {str(e)}")

def get_samples_in_last_week(url, token, sample_entry, start_ts=None, end_ts=None):
    """Get samples within the specified date range"""
    query = {
        'owner': 'visible',
        'query': {
            "entry_create_time": {
                "gte": int(start_ts * 1000) if start_ts else None,
                "lte": int(end_ts * 1000) if end_ts else None
            },
            "entry_type": sample_entry
        },
        'pagination': {
            'page_size': 10000
        }
    }
    
    print(f"Querying from {datetime.fromtimestamp(start_ts)} to {datetime.fromtimestamp(end_ts)}")
    response = requests.post(f"{url}/entries/query", json=query, 
                           headers={"Authorization": f"Bearer {token}"})
    return [sid["results"]["eln"]["lab_ids"][0] for sid in response.json()["data"]]

def get_entryids(url, token, sample_ids):  # give it a batch id
    # get al entries related to this batch id
    query = {
        'required': {
            'metadata': '*'
        },
        'owner': 'visible',
        'query': {'results.eln.lab_ids:any': sample_ids},
        'pagination': {
            'page_size': 100
        }
    }
    response = requests.post(
        f'{url}/entries/query', headers={'Authorization': f'Bearer {token}'}, json=query)
    response.raise_for_status()
    data = response.json()["data"]
    return [entry["entry_id"] for entry in data]

def get_specific_data_of_sample(url, token, sample_ids, entry_type):
    # collect the results of the sample, in this case it are all the annealing temperatures
    entry_ids = get_entryids(url, token, sample_ids)
    
    query = {
        'required': {
            'data': '*',
        },
        'owner': 'visible',
        'query': {'entry_references.target_entry_id:any': entry_ids,'section_defs.definition_qualified_name':entry_type},
        'pagination': {
            'page_size': 100
        }
    }
    response = requests.post(f'{url}/entries/archive/query',
                             headers={'Authorization': f'Bearer {token}'}, json=query)
    response.raise_for_status()
    linked_data = response.json()["data"]
    return linked_data

def parse_date_from_batch(batch_id):
    """Extract date from batch ID in format 'KIT_JoDa_20250406_1_0_10'"""
    try:
        date_str = batch_id.split('_')[2]
        return datetime.strptime(date_str, '%Y%m%d')
    except:
        return None

def extract_person_from_batch(batch_id):
    """Extract person identifier from batch ID (e.g., 'JoDa' from 'KIT_JoDa_20250406_1_0_10')"""
    try:
        name_part = batch_id.split('_')[1]
        if len(name_part) >= 4 and name_part[0].isupper() and name_part[1].islower() and name_part[2].isupper() and name_part[3].islower():
            return name_part
    except:
        pass
    return None

def update_plot(*args):
    with plot_output:
        clear_output()
        active_devices = {toggle.description for toggle in device_toggles.children 
                         if toggle.value}
        
        # Get date range for filtering
        plot_start_date = start_date.value
        plot_end_date = end_date.value
        
        fig = go.Figure()
        person_data = {}
        
        for batch_id, data in results.items():
            # Extract date first to filter early
            sample_date = parse_date_from_batch(batch_id)
            if not sample_date or sample_date < plot_start_date or sample_date > plot_end_date:
                continue
                
            person = extract_person_from_batch(batch_id)
            if not person:
                continue
                
            if person not in person_data:
                person_data[person] = []
            
            # Process each sample in the batch
            for sample_id, sample_data in data['samples'].items():
                # Skip samples without JV data
                if not sample_data['jvs']:
                    continue
                
                # Check if sample used any inactive devices
                sample_devices = sample_data['devices']
                if not sample_devices.issubset(active_devices):
                    # Skip if sample used any deactivated device
                    continue
                
                # Get maximum efficiency for valid measurements
                try:
                    max_efficiency = max(
                        curve['efficiency']
                        for jv in sample_data['jvs']
                        for curve in jv['jv_curve']
                        if 'efficiency' in curve
                    )
                    
                    person_data[person].append({
                        'date': sample_date,
                        'efficiency': max_efficiency,
                        'sample_id': sample_id,
                        'devices': list(sample_devices)
                    })
                except (ValueError, KeyError):
                    # Skip samples with no valid efficiency data
                    continue
        
        # Plot data by person
        for person, samples in person_data.items():
            if not samples:  # Skip persons with no valid data
                continue
                
            dates = [s['date'] for s in samples]
            efficiencies = [s['efficiency'] for s in samples]
            hover_text = [
                f"Sample: {s['sample_id']}<br>" +
                f"Efficiency: {s['efficiency']:.2f}%<br>" +
                f"Devices: {', '.join(s['devices'])}"
                for s in samples
            ]
            
            fig.add_scatter(
                x=dates,
                y=efficiencies,
                mode='markers',
                name=person,
                hovertext=hover_text,
                hoverinfo='text'
            )
        
        fig.update_layout(
            title='JV Measurements by Date and Person',
            xaxis_title='Date',
            yaxis_title='Efficiency (%)',
            showlegend=True
        )
        fig.show()

@login_output.capture()
def on_login_clicked(b):
    try:
        global token, results, samples, depositions, jvs
        print("Logging in...")  # Add feedback
        token = get_token_interactive(url, username.value, password.value)
        print("Successfully logged in, fetching data...")  # Add feedback
        
        # Get data using date range
        start_ts = start_date.value.timestamp()
        end_ts = end_date.value.timestamp()
        
        samples = get_samples_in_last_week(url, token, sample_entry, start_ts, end_ts)
        depositions = get_specific_data_of_sample(url, token, samples, "baseclasses.LayerDeposition")
        jvs = get_specific_data_of_sample(url, token, samples, "baseclasses.solar_energy.jvmeasurement.JVMeasurement")
        
        # Process results with new structure
        results = {}
        
        # First, organize depositions by sample
        for p in depositions:
            batch_samples = p["archive"]["data"]["samples"]
            for sample in batch_samples:
                try:
                    sample_id = sample["lab_id"]
                    batch_id = '_'.join(sample_id.split('_')[:-2])  # Remove sample numbering
                    
                    if batch_id not in results:
                        results[batch_id] = {'samples': {}}
                    
                    if sample_id not in results[batch_id]['samples']:
                        results[batch_id]['samples'][sample_id] = {
                            'devices': set(),
                            'jvs': []
                        }
                    
                    # Add device location if valid
                    if 'location' in p["archive"]["data"] and p["archive"]["data"]["location"]:
                        results[batch_id]['samples'][sample_id]['devices'].add(
                            p["archive"]["data"]["location"]
                        )
                    
                except KeyError as e:
                    print(f"KeyError in deposition: {e}")

        # Then add JV data
        for p in jvs:
            batch_samples = p["archive"]["data"]["samples"]
            for sample in batch_samples:
                try:
                    sample_id = sample["lab_id"]
                    batch_id = '_'.join(sample_id.split('_')[:-2])
                    
                    if batch_id in results and sample_id in results[batch_id]['samples']:
                        results[batch_id]['samples'][sample_id]['jvs'].append(p["archive"]["data"])
                        
                except KeyError as e:
                    print(f"KeyError in JV: {e}")
        
        # Remove the old sorting code that's no longer needed
        # Create device toggles from all unique devices
        devices = set()
        for batch_data in results.values():
            for sample_data in batch_data['samples'].values():
                devices.update(sample_data['devices'])
        
        device_toggles.children = [widgets.ToggleButton(
            value=True,
            description=device,
            layout=widgets.Layout(width='200px')
        ) for device in sorted(devices) if device]  # Filter out None/empty devices
        
        for toggle in device_toggles.children:
            toggle.observe(update_plot, 'value')
        
        # Update observers
        start_date.observe(update_plot, 'value')
        end_date.observe(update_plot, 'value')
        update_plot()
        
    except Exception as e:
        print(f"Error during login or data processing: {str(e)}")
        raise  # Re-raise to see full traceback during development

# Create and display the main UI container
main_container = widgets.VBox([
    widgets.HBox([username, password, login_button]),
    login_output,
    date_container,
    widgets.HTML("<h3>Select Devices:</h3>"),
    device_toggles,
    plot_output
])

# Connect the login handler and display interface
login_button.on_click(on_login_clicked)
display(main_container)